# Reversible Network Reconnection Model

Attempt at implementing the RNR model as described in Okuda et al. 2012:

>_Reversible network reconnection model for simulating large deformation in dynamic tissue morphogenesis_, Satoru Okuda, Yasuhiro Inoue, Mototsugu Eiraku, Yoshiki Sasai and Taiji Adachi Biomech Model Mechanobiol (2013) 12:627–644 [DOI 10.1007/s10237-012-0430-7](http://link.springer.com/article/10.1007%2Fs10237-012-0430-7)


The authors give 4 conditions which we detail and implement bellow.




## Condition 1 - Center of a face

The center of a face is defined by the average position of the face's edges midpoints, weighted by their lengths.

$$ \mathbf{r}_{\alpha} = \frac{\sum_{ij\alpha}\ell_{ij} (\mathbf{r}_i + \mathbf{r}_j)/2}{\sum_{ij\alpha}\ell_{ij}}$$


In [20]:
import numpy as np, pandas as pd
import ipyvolume as ipv
import matplotlib.pyplot as plt
%matplotlib inline

from tyssue import Epithelium
from tyssue import BulkGeometry, RNRGeometry, SheetGeometry
from tyssue import Sheet
from tyssue.utils import to_nd

from tyssue.config.geometry import bulk_spec
from tyssue.generation import three_faces_sheet
from tyssue.generation import extrude
from tyssue.draw import sheet_view, highlight_cells

from tyssue.topology.bulk_topology import IH_transition, HI_transition
from tyssue.topology import find_rearangements

from tyssue.dynamics import model_factory, effectors
from tyssue.solvers import QSSolver

draw_spec = {'face': {'visible': True}}




sheet = Sheet.planar_sheet_3d('sheet', 5, 6, 1, 1)
sheet.sanitize(trim_borders=True)

geom = RNRGeometry()
datasets = extrude(sheet.datasets, method='translation')

specs = bulk_spec()
eptm = Epithelium('20faces_3D', datasets, bulk_spec())
RNRGeometry.update_all(eptm)
RNRGeometry.center(eptm)
RNRGeometry.update_all(eptm)


eptm.settings['threshold_length'] = 1e-1
print(eptm.Nf, eptm.Ne, eptm.Nv)
draw_spec['face']['color'] = eptm.face_df.area
draw_spec['face']['color_range'] = (0, 1)


class NormPerimeterElasticity(effectors.PerimeterElasticity):
   
    @staticmethod
    def gradient(eptm):
        
        grad_srce_, grad_trgt_ = effectors.PerimeterElasticity.gradient(eptm)
        ns = to_nd(eptm.upcast_face(eptm.face_df.num_sides).to_numpy(), eptm.dim)
        grad_srce, grad_trgt = grad_srce_ * ns/6, grad_trgt_ * ns/6
        return grad_srce, grad_trgt
    
    @staticmethod
    def energy(eptm):
        
        ns = eptm.face_df.num_sides.mean()
        return effectors.PerimeterElasticity.energy(eptm) * ns / 6
  
        

effs = [
    NormPerimeterElasticity,
    effectors.FaceAreaElasticity,
    effectors.CellVolumeElasticity
]
model = model_factory(effs)


model_specs =  {
    'cell': {
        'is_alive': 1,
        'vol': 1.0,
        'vol_elasticity': 0.1,
        'prefered_vol':  6,
    },
    'face': {
        'is_alive': 1,
        'area': 1.0,
        'area_elasticity': 1.0,
        'prefered_area': 1.0,
        'perimeter': 1.0,
        'perimeter_elasticity': 0.1,
        'prefered_perimeter': 3.81
    },
        'edge': {},
        'vert': {},
        'settings': {'nrj_norm_factor': 1.0}
}

eptm.update_specs(model_specs, reset=True)
RNRGeometry.update_all(eptm)
solver = QSSolver()

solver.find_energy_min(eptm, geom, model)

ipv.clear()
fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

110 468 60


Figure(camera=PerspectiveCamera(fov=45.0, position=(0.0, 0.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0,…

## I→H transition

![IH transition in a bulk and a RNR](data/png/IH_transition.png)

This produces a shape similar to the one studied by LM Escudero and collegues [in their article](https://www.nature.com/articles/s41467-018-05376-1.pdf).





In [21]:
e_1011 = 300


eptm.settings['threshold_length'] = 0.5

srce, trgt, face, cell = eptm.edge_df.loc[e_1011,
                                          ['srce', 'trgt',
                                           'face', 'cell']]

IH_transition(eptm, e_1011)
RNRGeometry.update_all(eptm)

highlight_cells(eptm, cell, reset_visible=True)
draw_spec['face']['color'] = eptm.face_df.area

_ = solver.find_energy_min(eptm, geom, model)

ipv.clear()
fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

Figure(camera=PerspectiveCamera(fov=45.0, position=(0.0, 0.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0,…

# H → I transition

In [22]:
fa = 93


HI_transition(eptm, fa)
RNRGeometry.update_all(eptm)

highlight_cells(eptm, cell, reset_visible=False)
ipv.clear()
draw_spec['face']['color'] = eptm.face_df.area

fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

Figure(camera=PerspectiveCamera(fov=45.0, position=(0.0, 0.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0,…

### Testing for I→H / H →I transition triggers

In [23]:
eptm.edge_df.length.min()

0.18569133439530414

In [24]:
#eptm.settings['threshold_length'] = 1e-3
eptm.settings['threshold_length'] = eptm.edge_df.length.min() + 1e-3

In [25]:

ih, hi = find_rearangements(eptm)

### Condition 3

This condition is satisfied if `eptm.settings['threshold_length']`
is well defined, i.e, small with respect to the unit length or the
average edge length.

### Condition 4

* (i)  Two edges never share two vertices simultaneously.

_In our half-edge architecture, this is not as straight forward. But I think the condition can
be reformulated as:_ Two edges **from the same face** never share two edges simultanously.
I don't know how to demonstrate this, but I think these are equivalent (to get two edges together, you need to "squeeze" one face between those two.

* (ii) Two polygonal faces never share two or more edges simultaneously.


In [26]:
from tyssue.topology.base_topology import condition_4i, condition_4ii

In [27]:
condition_4i(eptm), condition_4ii(eptm)

(Int64Index([], dtype='int64', name='face'),
 array([], shape=(0, 2), dtype=int64))

In [28]:

IH_transition(eptm, ih[0])
RNRGeometry.update_all(eptm)
solver.find_energy_min(eptm, RNRGeometry, model)
draw_spec['face']['color'] = eptm.face_df.area

ipv.clear()
fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

Figure(camera=PerspectiveCamera(fov=45.0, position=(0.0, 0.0, 2.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0,…

In [29]:
assert eptm.validate()